In [ ]:
import gc
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import joblib
from tqdm.notebook import tqdm

import os
import riiideducation

### This kernel is written because reading pkl(type is dict) will used soooooooo exaggerated RAM.

# method 1

1. Create several required ndarray (there is 3)
2. Dict just store index of ndarray
3. read them now


In [ ]:
# one dict
curr_u_dict = pickle.load(open("../input/merge-data1615/curr_u_dict_1615.pkl.data","rb")) # dict

# three ndarray
np_cor_cnt = pickle.load(open("../input/merge-data1615/np_cor_cnt_1615.pkl.data","rb")) # ndarray
np_all_cnt = pickle.load(open("../input/merge-data1615/np_all_cnt_1615.pkl.data","rb")) # ndarray
np_uctdiff_cnt = pickle.load(open("../input/merge-data1615/np_uctdiff_cnt_1615.pkl.data","rb")) # ndarray

The following is the function used with the dict and ndarray when testing

In [ ]:
def add_user_feats_without_update(df):
    ucc = np.zeros(len(df), dtype=np.uint16)
    uac = np.zeros(len(df), dtype=np.uint16)
    uqcor = np.zeros(len(df), dtype=np.uint8)
    uqcnt = np.zeros(len(df), dtype=np.uint8)
    utdiff = np.zeros(len(df), dtype=np.uint64)
    utdiff_mean = np.zeros(len(df), dtype=np.uint64) 
    uelapdiff = np.zeros(len(df), dtype=np.float32)  
    uq_timediff = np.zeros(len(df), dtype=np.uint64) 
    for cnt,row in enumerate(df[['user_id', 'content_id','timestamp','prior_question_elapsed_time']].itertuples(index=False)): 
        if row[0] in curr_u_dict:
            ucc[cnt] = curr_u_dict[row[0]]["ucc"]
            uac[cnt] = curr_u_dict[row[0]]["uac"]
            utdiff[cnt] = row[2] - curr_u_dict[row[0]]["uts"]
            utdiff_mean[cnt] = curr_u_dict[row[0]]["utsdiff"][1] / curr_u_dict[row[0]]["utsdiff"][0] 
            uelapdiff[cnt] = row[3] - curr_u_dict[row[0]]["uelapdiff"] 
            if row[1] in curr_u_dict[row[0]]:
                tmp_idx = curr_u_dict[row[0]][row[1]]
                uq_timediff[cnt] =  row[2] - np_uctdiff_cnt[tmp_idx] 
                uqcor[cnt] = np_cor_cnt[tmp_idx]
                uqcnt[cnt] = np_all_cnt[tmp_idx]
            else:
                uqcor[cnt] = 0; uqcnt[cnt] = 0
                uq_timediff[cnt] = 0 
        else:
            ucc[cnt] = 0; uac[cnt] = 0
            uqcor[cnt] = 0; uqcnt[cnt] = 0
            utdiff[cnt] = 0; utdiff_mean[cnt] = 0; 
            uelapdiff[cnt] = 0; uq_timediff[cnt] = 0 
            
    user_feats_df = pd.DataFrame({'curr_user_correct_cnt':ucc, 'curr_user_answer_cnt':uac,
                                  'curr_uq_correct_cnt':uqcor, 'curr_uq_answer_cnt':uqcnt,
                                  'curr_user_time_diff':utdiff, 'curr_user_time_diff_mean':utdiff_mean, 
                                  'curr_user_elapsed_time_diff':uelapdiff, 'curr_uq_time_diff':uq_timediff 
                                 }) 
    user_feats_df['curr_uq_acc'] = user_feats_df['curr_uq_correct_cnt'] / user_feats_df['curr_uq_answer_cnt']
    user_feats_df['curr_uq_acc'].fillna(0.680, inplace=True)
    user_feats_df['curr_uq_acc'] = user_feats_df['curr_uq_acc'].astype(np.float32)
    user_feats_df['curr_uq_correct_cnt'] = user_feats_df['curr_uq_correct_cnt'].where(user_feats_df['curr_uq_correct_cnt'] <= 4, 4)
    user_feats_df['curr_uq_answer_cnt'] = user_feats_df['curr_uq_answer_cnt'].where(user_feats_df['curr_uq_answer_cnt'] <= 4, 4)
    user_feats_df['curr_user_acc'] = user_feats_df['curr_user_correct_cnt'] / user_feats_df['curr_user_answer_cnt']
    user_feats_df['curr_user_acc'].fillna(0.680, inplace=True)
    user_feats_df['curr_user_acc'] = user_feats_df['curr_user_acc'].astype(np.float32)
    user_feats_df['curr_user_time_diff_mean'].fillna(0, inplace=True)  
    user_feats_df['curr_user_elapsed_time_diff'].fillna(0, inplace=True) 
    user_feats_df['curr_uq_time_diff'].fillna(0, inplace=True)  
    df = pd.concat([df, user_feats_df], axis=1)
    return df


def update_user_feats(df):
    global idx
    for row in df[['user_id','content_id','answered_correctly','timestamp', 'content_type_id','prior_question_elapsed_time',]].values: 
        if row[4] == 0:
            if row[0] in curr_u_dict:
                curr_u_dict[row[0]]["ucc"] += row[2]
                curr_u_dict[row[0]]["uac"] += 1
                curr_u_dict[row[0]]["uts"] = row[3]
                curr_u_dict[row[0]]["utsdiff"][0] += 1 
                curr_u_dict[row[0]]["utsdiff"][1] += row[3] 
                curr_u_dict[row[0]]["uelapdiff"] = row[5] 
                if row[1] in curr_u_dict[row[0]]:
                    tmp_idx = curr_u_dict[row[0]][row[1]]
                    np_uctdiff_cnt[tmp_idx] = row[3] 
                    np_cor_cnt[tmp_idx] += row[2]
                    np_all_cnt[tmp_idx] += 1
                else:
                    curr_u_dict[row[0]][row[1]] = idx
                    np_uctdiff_cnt[idx] = row[3] 
                    np_cor_cnt[idx] += row[2]
                    np_all_cnt[idx] += 1
                    idx += 1
            else:
                curr_u_dict[row[0]] = {}
                curr_u_dict[row[0]]["ucc"] = row[2]
                curr_u_dict[row[0]]["uac"] = 1
                curr_u_dict[row[0]]["uts"] = row[3]
                curr_u_dict[row[0]]["utsdiff"] = [1, row[3]] 
                curr_u_dict[row[0]]["uelapdiff"] = row[5] 
                curr_u_dict[row[0]][row[1]] = idx
                np_uctdiff_cnt[idx] = row[3] 
                np_cor_cnt[idx] += row[2]
                np_all_cnt[idx] += 1
                idx += 1

# method 2

1. convert the dictionary into a pandas dataframe.(pandas.to_dict)
2. save it and then upload it to kaggle.
3. read it by writing your own function, as shown below.
4. Of course pandas has pandas.from_dict ,but it does not apply to the case of multiple nesting.(like me).

In [ ]:
# 
def to_letures_dict(df):
    da = {}
    for t in tqdm(df.itertuples(name=None), total=df.shape[0]):
        key = t[0]
        sub_dict = {}
        for i, col in enumerate(df.columns, 1):
            if col == "has_tags":
                sub_dict[col] = eval(t[i])
            else:
                sub_dict[col] = t[i]
        da[key] = sub_dict
    return da

curr_lectures_dict = pd.read_csv("../input/merge-data1615/curr_lectures_dict_1615.csv.data", index_col=0)
curr_lectures_dict = to_letures_dict(curr_lectures_dict)

The following is the function used with the dict and ndarray when testing

In [ ]:
lectures_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
lectures_df['type_of'] = lectures_df['type_of'].replace('solving question', 'solving_question')
lectures_df = pd.get_dummies(lectures_df, columns=['part', 'type_of'])
lectures_df['content_type_id'] = 1

q_taglist_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/questions.csv")[['tags']].astype(str)
q_taglist_df["tags_l"] = [x.split() for x in q_taglist_df.tags.values]
q_taglist_df['content_type_id'] = 0
q_taglist_df.drop("tags", axis=1, inplace=True)
q_taglist_df.drop(10033, axis=0, inplace=True) # nan

def add_lectures_feats(df, curr_dict):
    new_df = df[["row_id", "user_id", "timestamp", "content_id", "content_type_id"]]
    new_df = new_df.merge(lectures_df, how="left", left_on = ["content_id","content_type_id"], right_on = ["lecture_id","content_type_id"])
    new_df = new_df.merge(q_taglist_df, how="left", left_on = ["content_id","content_type_id"], right_on = [q_taglist_df.index,"content_type_id"])
    new_df = new_df.sort_values(["timestamp"])
    new_df = new_df[['timestamp', 'user_id', 'content_type_id','tag','part_1','part_2','part_3','part_4','part_5','part_6','part_7',
                     'type_of_concept','type_of_intention','type_of_solving_question','type_of_starter','tags_l','row_id']]
    ulc_lb = np.zeros(len(df), dtype="int8")
    part1_l = np.zeros(len(df), dtype="uint16")
    part2_l = np.zeros(len(df), dtype="uint16")
    part3_l = np.zeros(len(df), dtype="uint16")
    part4_l = np.zeros(len(df), dtype="uint16")
    part5_l = np.zeros(len(df), dtype="uint16")
    part6_l = np.zeros(len(df), dtype="uint16")
    part7_l = np.zeros(len(df), dtype="uint16")
    type_of_concept_l = np.zeros(len(df), dtype="uint16")
    type_of_intention_l = np.zeros(len(df), dtype="uint16")
    type_of_solving_question_l = np.zeros(len(df), dtype="uint16")
    type_of_starter_l = np.zeros(len(df), dtype="uint16")
    has_tags_l = np.zeros(len(df), dtype="float32")
    
    # 0.'timestamp', 1.'user_id', 2.'content_type_id',3.'tag',4.'part_1',5.'part_2',6.'part_3',7.'part_4',8.'part_5',9.'part_6',10.'part_7',
    # 11.'type_of_concept',12.'type_of_intention',13.'type_of_solving_question',14.'type_of_starter',15.'tags_l', 16.'row_id'
    for cnt,row in enumerate(new_df.itertuples(index=False)):
        if row[1] in curr_dict:
            if row[2] == 1:
                curr_dict[row[1]]["lecture_bool"] = 1
                curr_dict[row[1]]["part_1_cnt"] += int(row[4])
                curr_dict[row[1]]["part_2_cnt"] += int(row[5])
                curr_dict[row[1]]["part_3_cnt"] += int(row[6])
                curr_dict[row[1]]["part_4_cnt"] += int(row[7])
                curr_dict[row[1]]["part_5_cnt"] += int(row[8])
                curr_dict[row[1]]["part_6_cnt"] += int(row[9])
                curr_dict[row[1]]["part_7_cnt"] += int(row[10])
                curr_dict[row[1]]["type_of_concept_cnt"] += int(row[11])
                curr_dict[row[1]]["type_of_intention_cnt"] += int(row[12])
                curr_dict[row[1]]["type_of_solving_question_cnt"] += int(row[13])
                curr_dict[row[1]]["type_of_starter_cnt"] += int(row[14])
                curr_dict[row[1]]["has_tags"].add(int(row[3]))
        else:
            curr_dict[row[1]] = {}
            if row[2] == 1:
                curr_dict[row[1]]["lecture_bool"] = 1
                curr_dict[row[1]]["part_1_cnt"] = int(row[4])
                curr_dict[row[1]]["part_2_cnt"] = int(row[5])
                curr_dict[row[1]]["part_3_cnt"] = int(row[6])
                curr_dict[row[1]]["part_4_cnt"] = int(row[7])
                curr_dict[row[1]]["part_5_cnt"] = int(row[8])
                curr_dict[row[1]]["part_6_cnt"] = int(row[9])
                curr_dict[row[1]]["part_7_cnt"] = int(row[10])
                curr_dict[row[1]]["type_of_concept_cnt"] = int(row[11])
                curr_dict[row[1]]["type_of_intention_cnt"] = int(row[12])
                curr_dict[row[1]]["type_of_solving_question_cnt"] = int(row[13])
                curr_dict[row[1]]["type_of_starter_cnt"] = int(row[14])
                curr_dict[row[1]]["has_tags"] = set([int(row[3])])
            else:
                curr_dict[row[1]]["lecture_bool"] = 0
                curr_dict[row[1]]["part_1_cnt"] = 0
                curr_dict[row[1]]["part_2_cnt"] = 0
                curr_dict[row[1]]["part_3_cnt"] = 0
                curr_dict[row[1]]["part_4_cnt"] = 0
                curr_dict[row[1]]["part_5_cnt"] = 0
                curr_dict[row[1]]["part_6_cnt"] = 0
                curr_dict[row[1]]["part_7_cnt"] = 0
                curr_dict[row[1]]["type_of_concept_cnt"] = 0
                curr_dict[row[1]]["type_of_intention_cnt"] = 0
                curr_dict[row[1]]["type_of_solving_question_cnt"] = 0
                curr_dict[row[1]]["type_of_starter_cnt"] = 0
                curr_dict[row[1]]["has_tags"] = set()
        
        ulc_lb[cnt] = curr_dict[row[1]]["lecture_bool"]
        part1_l[cnt] = curr_dict[row[1]]["part_1_cnt"]
        part2_l[cnt] = curr_dict[row[1]]["part_2_cnt"]
        part3_l[cnt] = curr_dict[row[1]]["part_3_cnt"]
        part4_l[cnt] = curr_dict[row[1]]["part_4_cnt"]
        part5_l[cnt] = curr_dict[row[1]]["part_5_cnt"]
        part6_l[cnt] = curr_dict[row[1]]["part_6_cnt"]
        part7_l[cnt] = curr_dict[row[1]]["part_7_cnt"]
        type_of_concept_l[cnt] = curr_dict[row[1]]["type_of_concept_cnt"]
        type_of_intention_l[cnt] = curr_dict[row[1]]["type_of_intention_cnt"]
        type_of_solving_question_l[cnt] = curr_dict[row[1]]["type_of_solving_question_cnt"]
        type_of_starter_l[cnt] = curr_dict[row[1]]["type_of_starter_cnt"]
        
        if type(row[15]) == list:
            tags_has = 0
            for tag in row[15]:
                if int(tag) in curr_dict[row[1]]["has_tags"]:
                    tags_has += 1
            has_tags_l[cnt] = tags_has/len(row[15])

    has_tags_lb = (has_tags_l > 0).astype("int8")

    lectures_feats_df = pd.DataFrame({"curr_lecture_bool":ulc_lb,
                                      "part_1_cnt":part1_l,
                                      "part_2_cnt":part2_l,
                                      "part_3_cnt":part3_l,
                                      "part_4_cnt":part4_l,
                                      "part_5_cnt":part5_l,
                                      "part_6_cnt":part6_l,
                                      "part_7_cnt":part7_l,
                                      "type_of_concept_cnt":type_of_concept_l,
                                      "type_of_intention_cnt":type_of_intention_l,
                                      "type_of_solving_question_cnt":type_of_solving_question_l,
                                      "type_of_starter_cnt":type_of_starter_l,
                                      "watched_tags_rate":has_tags_l,
                                      "watched_tags_bool":has_tags_lb,
                                     }).set_index(new_df["row_id"])

    df = df.merge(lectures_feats_df,how="left",left_on="row_id",right_index=True)
    return df

# Testing


In [ ]:
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
previous_test_df = None
for (test_df, sample_prediction_df) in iter_test:
    if previous_test_df is not None:
        previous_test_df[target] = eval(test_df["prior_group_answers_correct"].iloc[0])
        update_user_feats(previous_test_df)  # use the function
        previous_test_df = previous_test_df[previous_test_df[target] != -1]
        
    previous_test_df = test_df.copy()
    
    test_df = add_lectures_feats(test_df, curr_lectures_dict) # use the function
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    test_df = add_user_feats_without_update(test_df) # use the function
    
    test_df = add_up_without_update(test_df) # use the function
    
    # ...balabalabala...(Other codes omitted)

    test_df['answered_correctly'] = model.predict(test_df[features])
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])